### Transform regression dataset

In [1]:
import torch
import torchvision

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder

from torchvision.transforms import v2

In [2]:
import os
import json

import numpy as np

from PIL import Image

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
class DatasetReg(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform

        self.list_name_file = os.listdir(path)
        if 'coords.json' in self.list_name_file:
            self.list_name_file.remove('coords.json')

        self.len_dataset = len(self.list_name_file)

        with open(os.path.join(self.path, 'coords.json'), 'r') as f:
            self.dict_coords = json.load(f)
        
    def __len__(self):
        return self.len_dataset

    def __getitem__(self, index):
        name_file = self.list_name_file[index]
        path_img = os.path.join(self.path, name_file)

        img = Image.open(path_img)
        coords = torch.tensor(self.dict_coords[name_file], dtype=torch.float32)

        if self.transform:
            img = self.transform(img)

        return img, coords

In [5]:
transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=(0.5, ), std=(0.5, ))
    ]
)

In [6]:
dataset = DatasetReg(os.path.join(os.getcwd(), 'dataset'), transform=transform)

In [7]:
# split data
train_data, val_data, test_data = random_split(dataset, [0.7, 0.1, 0.2])

In [8]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [9]:
imgs, coord = next(iter(train_loader))

print('Imgs:')
print(f'    {type(imgs)}')
print(f'    {imgs.shape}')
print(f'    {imgs.dtype}')
print(f'    min = {imgs.min()}, max = {imgs.max()}')
print('coord:')
print(f'    {type(coord)}')
print(f'    {coord.shape}')
print(f'    {coord.dtype}')

Imgs:
    <class 'torch.Tensor'>
    torch.Size([64, 1, 64, 64])
    torch.float32
    min = -1.0, max = 0.7960785627365112
coord:
    <class 'torch.Tensor'>
    torch.Size([64, 2])
    torch.float32


In [10]:
imgs, cls = next(iter(train_loader))

print('Imgs:')
print(f'    {type(imgs)}')
print(f'    {imgs.shape}')
print(f'    {imgs.dtype}')
print(f'    min = {imgs.min()}, max = {imgs.max()}')
print('cls:')
print(f'    {type(cls)}')
print(f'    {cls.shape}')
print(f'    {cls.dtype}')

Imgs:
    <class 'torch.Tensor'>
    torch.Size([64, 1, 64, 64])
    torch.float32
    min = -1.0, max = 0.8196079730987549
cls:
    <class 'torch.Tensor'>
    torch.Size([64, 2])
    torch.float32
